# Custom Models, Layers, and Loss Functions with TensorFlow 
### *Week: 3*

#### DeepLearningAI

## Description

* Compare Functional and Sequential APIs, discover new models you can build with the Functional API, and build a model that produces multiple outputs including a Siamese network.
* Build custom loss functions (including the contrastive loss function used in a Siamese network) in order to measure how well a model is doing and help your neural network learn from training data. 
* Build off of existing standard layers to create custom layers for your models, customize a network layer with a lambda layer, understand the differences between them, learn what makes up a custom layer, and explore activation functions. 
* Build off of existing models to add custom functionality, learn how to define your own custom class instead of using the Functional or Sequential APIs, build models that can be inherited from the TensorFlow Model class, and build a residual network (ResNet) through defining a custom model class. 


The DeepLearning.AI TensorFlow: Advanced Techniques Specialization introduces the features of TensorFlow that provide learners with more control over their model architecture and tools that help them create and train advanced ML models.  

This Specialization is for early and mid-career software and machine learning engineers with a foundational understanding of TensorFlow who are looking to expand their knowledge and skill set by learning advanced TensorFlow features to build powerful models.

## Custom Lambda Layer

### Introduction to Lambda Layers

>**Lambda Layer**:
>This is a layer type that can be used to execute arbitrary code. The purpose of the lambda layer, like I said, is to execute an arbitrary function within a sequential or a functional API model. It's best-suited for something quick and simple or if you want to experiment.

The simplest Lambda Layer

```Python
tf.keras.layers.Lambda(lambda x: tf.abs(x))
```

#### We could change 'relu' layer to lambda layer in below code

```Python
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])


# Same loss and accuracy values with this

mode = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape(28, 28)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(lambda x: tf.abs(x)),
    tf.keras.layers.Dense(10, activation='softmax')
])
```

### Custom Functions from Lambda Layers

>Previously you saw how to use Lambda layers to execute arbitrary code within your layer definition. Another example is to have a custom function that the Lambda layer can call in order to encapsulate your code. So for example, if you wanted to implement a modified Relu with a threshold, you could do so. 


```Python
def my_relu(x):
    return tf.max(0.0, x) # You can tweak yout relu function.

model = tf.keras.layers.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(my_relu)
    tf.keras.layers.Dense(10, activation='softmax')
])

```

### [The first lab notebook](C1_W3_Lab_1_lambda-layer.ipynb)

## Implementing Custom Layers

### Architecture of a Custom Layer

**Some commonly used layers**

|Convolutional|Recurrent|Pooling|Merge|Activations|Core|
|--|--|--|--|--|--|
|Conv1D, Conv2D, Conv3D|LSTM|MaxPooling2D|Add|LeakyReLU|Activation|
|SeperableConv2D|GRU|AveragePooling2D|Subtract|PReLU|Lambda|
|DepthwiseConv2D| |GlobalAveragePooling2D|Multiply|ELU|Input|
||||||Dense|
||||||Dropout|
||||||BatchNormalization|




### What is a Layer?
![A-layer](what_is_layer.png)

>Typically it's a class that collects parameters that encapsulates state and computation to achieve the layers purpose in a neural network. Whether you're using it in the sequential or functional API, every model architecture design item is a layer. 

>When we say states, consider this to be a variable, something that makes a particular instance of a layer unique. These variables can be trainable where during model.fit, TensorFlow can tweak their values to test for better performance or they can be non-trainable, where they might be used for some other feature. 

>Computation is the means of transforming a batch of inputs into a batch of outputs. It's typically called the forward pass in neural networks, where a calculation is made and then pass to the next layer

```Python
class SimpleDense(Layer):
    
    def __init__(self, units=25):
        super(SimpleDense, self).__init__()
        self.units = units
        
    def build(selfi input_shape): # Create the state of the layer (weights)
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units), dtype='float32'),
            trainable=True
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            name="bias",
            initial_value = b_init(shape=(self.units,), dtype='float32'),
            trainable=True)
        
    def call(self, inputs): # Defines the computation from inputs to outputs
        return tf.matmul(inputs, self.w) + self.b
    
    
# In action

my_dense = SimpleDense(units=1)
x = tf.ones((1, 1))
y = my_dense(x)
print(my_dense.variables)
```

### Train a neural network with your Custom Layer

The hello world nn.

```Python
import numpy as np

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model = tf.keras.Sequential([SimpleDense(units=1)])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500, verbose=0)
print(model.predict([10.0]))
```


**Change fashion mnist code with our newest code**

##### OLD

```Python
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])
```
##### NEW

```Python
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape(28, 28)),
    SimpleDense(128),
    tf.keras.layers.Lambda(my_relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])
```

### [The second lab notebook](C1_W3_Lab_2_custom-dense-layer.ipynb)

## Activation Custom Layers

>ou saw how these can be the basic building blocks of a deep neural network by being able to add them in layers, and you also saw how to build some simple machine learning models using them. But your custom layer was missing the ability to do an activation on them. There was a workaround that we did using a lambda layer, but it's simpler and cleaner to specify an activation function on a layer. Let's take a look at how to expand our dense class to be able to do that. 

#### Change Basic Mnist Model

```Python
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape(28, 28)),
    SimpleDense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])
```

*Customize our **Simple Dense* layer for activation*

```Python
class SimpleDense(Layer):
    
    def __init__(self, units=25, activation=None):
        super(SimpleDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
        
    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.w) + self.b)
```

### [The third lab notebook](C1_W3_Lab_3_custom-layer-activation.ipynb)